# Bot для торговли на бирже Binance

https://github.com/binance/binance-futures-connector-python

Стратегия торговли: вход в сделку на основе предсказаний модели обученной на данных стакана.

## Импорт библиотек

In [94]:
from binance.client import Client
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging
import pandas as pd
import numpy as np
import talib
import keys
import time
from datetime import datetime
from pytz import timezone
import logging
import requests
import json
import re
pd.options.mode.chained_assignment = None # отключение дебильного предупреждения
pd.set_option('display.max_columns', None) # показывать максимальное количесвто столбцов

## Подключение к бирже

In [95]:
def connecting_to_market():
    try:
        client_alex = UMFutures(keys.api_key, keys.secret_key)
        result_alex = f'Подключение client_alex к бирже выполено.'
    except:
        result_alex = f'client_alex: ошибка при подключении к бирже.' 

    try:
        client_den = UMFutures(keys.api_key_den, keys.secret_key_den)
        result_den = f'Подключение client_den к бирже выполено.'
    except:
        result_den = f'client_den: ошибка при подключении к бирже.'

    return client_alex, result_alex, client_den, result_den
        

In [96]:
clients = connecting_to_market()
client, client_den = clients[0], clients[2]

## Задать переменные

In [97]:
BOT_NAME = '📈'
SYMBOL = 'MOODENGUSDT'
WINDOW_MA = 50 # окно для рассчета MA
TREND_THRESHOLD = 0.9994 # порог силы тренда
PAUSE_BETWEEN_GET_DATA_IN_POSITION = 10 # период между обновлением данных в секундах
TP_1_PROC = 0.5 #  это проценты
TP_2_PROC = 1.5 
TP_3_PROC = 2.0
TP_4_PROC = 2.5
STOP_LOSS_PROC = 1.0   #  это проценты
POS_SIZE = 100.0 # размер позиции в менетах
ZONE = timezone('Europe/Moscow') # часовой пояс

## Запуск логирования

In [98]:
def start_logging(Bot_name):
    """
    Bot_name - имя бота - str
    """
    logger = logging.getLogger(Bot_name)
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler('bot.log')
    fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(fh)
    return logger

In [99]:
#logger = start_logging(BOT_NAME)

In [100]:
#logger.warning('LLLLL')

## Получение данных

### REST_API

#### Стакан

In [101]:
# client.depth(SYMBOL, limit = 10)

In [102]:
df = pd.DataFrame(client.depth(SYMBOL, limit = 10))

In [103]:
df.head(10)

,lastUpdateId,E,T,bids,asks
0,7894393410702,1751010136153,1751010136138,"[0.1355500, 3472]","[0.1355600, 3781]"
1,7894393410702,1751010136153,1751010136138,"[0.1355400, 7608]","[0.1355700, 3668]"
2,7894393410702,1751010136153,1751010136138,"[0.1355300, 13616]","[0.1355800, 9759]"
3,7894393410702,1751010136153,1751010136138,"[0.1355200, 26047]","[0.1355900, 18533]"
4,7894393410702,1751010136153,1751010136138,"[0.1355100, 46603]","[0.1356000, 35731]"
5,7894393410702,1751010136153,1751010136138,"[0.1355000, 57118]","[0.1356100, 26487]"
6,7894393410702,1751010136153,1751010136138,"[0.1354900, 23435]","[0.1356200, 30128]"
7,7894393410702,1751010136153,1751010136138,"[0.1354800, 48509]","[0.1356300, 38727]"
8,7894393410702,1751010136153,1751010136138,"[0.1354700, 31872]","[0.1356400, 32700]"
9,7894393410702,1751010136153,1751010136138,"[0.1354600, 34706]","[0.1356500, 36083]"


### WebSockets

#### Функции для подключения

In [104]:
def start_Socket(handler):
    '''
    Запуск сокета
    '''
    my_client = UMFuturesWebsocketClient(on_message = handler)
    try:
        my_client.diff_book_depth(
        symbol="moodengusdt",
        speed=500,
        id=1)
        time.sleep(43200)
        logging.debug("closing ws connection")
        my_client.stop()
        
    except:
        time.sleep(2)
        logging.debug("connection restart")
        my_client.diff_book_depth(
        symbol="moodengusdt",
        speed=500,
        id=1)
        time.sleep(15000)
        logging.debug("closing ws connection")
        my_client.stop()


In [105]:
data = []

In [106]:
def handler_debug(_, message):
    """
    Вывод всей инфы стакана
    """
    if json.loads(message)['e'] == 'depthUpdate':
        data.append(message)
        print(message)

In [107]:
def handler_orderbook_update(_, message):
    """
    Предобработка данных стакана
    """
    if json.loads(message)['e'] == 'depthUpdate':
        try:
            bids = [float(b[1]) for b in json.loads(message)['b']][::-1][:10]
            asks = [float(a[1]) for a in json.loads(message)['a']][:10]
            mid_price = [(float(json.loads(message)['a'][-1][0]) + float(json.loads(message)['b'][::-1][0][0])) / 2] 
            total_feat = mid_price + asks + bids 
            df_row = pd.DataFrame(total_feat).T
            df_row.columns = ['mid_price', 'ask_1', 'ask_2', 'ask_3', 'ask_4', 'ask_5', 'ask_6', 'ask_7', 'ask_8', 'ask_9', 'ask_10', 
                        'bid_1', 'bid_2', 'bid_3', 'bid_4', 'bid_5', 'bid_6', 'bid_7', 'bid_8', 'bid_9', 'bid_10']
            data.append(df_row)
            #print(message)
        except:
            df_row = pd.DataFrame()  
            data.append(df_row)
            print(message)  

#### Запуск сбора данных

In [108]:
#start_Socket(handler_orderbook_update)

#### Получение датафрейма

In [109]:
#ds_ws = pd.concat(data).reset_index(drop=True)

In [110]:
#ds_ws.shape

In [111]:
#ds_ws

#### Выгрузка датафрейма

In [112]:
#ds_ws.to_csv('ds_moodeng_13_06.csv', index = False)